## Goal of notebook

    Look at boundary conditions for west, north
    parametrize according to salinity relationships for JdF and northern strait from this notebook:
    /data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/INITIALIZING_OXYGEN
    /data/tjarniko/MEOPAR/at3/notebooks/carbon_dev/INITIALIZING_CARBON
    
    make a new init file
    
    where does open_boundaries lead to? it must be:
        /home/tjarnik/MEOPAR/tracers/north
            bioOBC_full2_north.nc
        /home/tjarnik/MEOPAR/tracers/west
    but where is the link?
            bioOBC_full2_west.nc
 
    
    
    relationships DIC (JdF, N, S, Haro):
    slopes:
    [68.984506591824541, 75.046682176725568, 45.248982910520937, 62.275868280536201]
    intercepts:
    [-100.13978357455562, -210.80778641253437, 669.0670004789929, 128.20407948614184]

    relationships alkalinity (JdF, N, S, Haro):
    slopes:
    [46.987083509452937, 41.703959570158162, 45.255106772714242, 48.099718320884534]
    intercepts:
    [666.6180299218554, 805.39142381007036, 712.25447700734412, 630.84821673141755]
    
    # relationships OXY (Jdf, N, S, Haro:)
    # slopes:
    # [-1.1093358620615621, -1.0324527832500823, -0.72610938589358809, -0.74540955174472656]
    # intercepts:
    # [39.482801413558569, 35.238673873674337, 25.996793064314204, 27.695179997365347]
    # ox at 26

currently initializing to NO3

In [2]:
#import all the things

%reset
import netCDF4 as nc
import matplotlib.pyplot as plt
import datetime
import os
import numpy as np
import cmocean as cm
from salishsea_tools import visualisations as vis
from salishsea_tools import (teos_tools, tidetools, viz_tools)
import xarray as xr 
import subprocess 
import cmocean

%matplotlib inline

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
d_nor = '/data/tjarniko/MEOPAR/tracers/north/'
f_nor = 'bioOBC_full2_north.nc'
phys_nor = 'SalishSea_north_TEOS10.nc'
d_wes = '/data/tjarniko/MEOPAR/tracers/west/'
f_wes = 'bioOBC_full2_west.nc'
phys_wes = 'SalishSea_west_TEOS10.nc'


tr_north = nc.Dataset(d_nor+f_nor)
tr_west = nc.Dataset(d_wes+f_wes)
phys_north = nc.Dataset(d_nor+phys_nor)
phys_west = nc.Dataset(d_wes+phys_wes)

In [4]:
print('tracers north')
print(tr_north)
print('tracers west')
print(tr_west)
print('physics north')
print(phys_north)
print('physics west')
print(phys_west)

tracers north
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): xbT(300), yb(1), deptht(40), time_counter(2)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT), float32 NH4(time_counter,deptht,yb,xbT), float32 DIA(time_counter,deptht,yb,xbT), float32 CRY(time_counter,deptht,yb,xbT), float32 MYRI(time_counter,deptht,yb,xbT), float32 MICZ(time_counter,deptht,yb,xbT), float32 PON(time_counter,deptht,yb,xbT), float32 DON(time_counter,deptht,yb,xbT), float32 bSi(time_counter,deptht,yb,xbT), float32 O2(time_counter,deptht,yb,xbT), float32 tur(time_counter,deptht,yb,xbT)
    groups: 

tracers west
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): de

In [5]:
sn_nor = phys_north.variables['vosaline']
print('northern salinity')
#print(sn_nor)
#print(sn_nor[0,2,:,:])
sn_wes = phys_west.variables['vosaline']
print('western salinity')
print(sn_wes)
print(sn_wes[0,2,:,0])

print(tr_west)

NO3 = tr_west.variables['NO3']

#print(NO3[0,2,:,0:870:87])

northern salinity
western salinity
<class 'netCDF4._netCDF4.Variable'>
float32 vosaline(time_counter, deptht, yb, xbT)
    coordinates: nav_lon nav_lat deptht time_counter
    grid: SalishSea
    units: g/kg
    long_name: Reference Salinity
unlimited dimensions: time_counter
current shape = (52, 40, 10, 87)
filling on, default _FillValue of 9.969209968386869e+36 used

[ 29.95800018  29.95899963  29.95999908  29.95100021  29.95400047
  29.95599937  29.95700073  29.95800018  29.95800018  29.95899963]
<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): deptht(40), yb(1), xbT(870), time_counter(52)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT), float32 NH4(time_counter,deptht,yb,xbT), float32 DIA(tim

In [6]:
dsin = tr_north

#output file
ncname = 'tr_northDICTAOXY.nc'
dsout = nc.Dataset(ncname, "w", format="NETCDF4")
#Copy dimensions
for dname, the_dim in dsin.dimensions.items():
    print(dname)
    print(len(the_dim))
    dsout.createDimension(dname, len(the_dim) if not the_dim.isunlimited() else None)
# Copy variables
for v_name, varin in dsin.variables.items():
    outVar = dsout.createVariable(v_name, varin.datatype, varin.dimensions)
    print(varin.datatype)
    
    # Copy variable attributes
    outVar.setncatts({k: varin.getncattr(k) for k in varin.ncattrs()})
    
    outVar[:] = varin[:]

xbT
300
yb
1
deptht
40
time_counter
2
float32
float32
float32
int32
int32
int32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32
float32


In [7]:
DIC = dsout.createVariable('DIC',np.float64,('time_counter','deptht','yb','xbT'))
TA = dsout.createVariable('TA',np.float64,('time_counter','deptht','yb','xbT'))
OXY = dsout.createVariable('OXY',np.float64,('time_counter','deptht','yb','xbT'))

In [8]:
# relationships DIC (JdF, N, S, Haro:)
#     slopes:
#     [68.984506591824541, 75.046682176725568, 45.248982910520937, 62.275868280536201]
#     intercepts:
#     [-100.13978357455562, -210.80778641253437, 669.0670004789929, 128.20407948614184]

#     relationships alkalinity (JdF, N, S, Haro):
#     slopes:
#     [46.987083509452937, 41.703959570158162, 45.255106772714242, 48.099718320884534]
#     intercepts:
#     [666.6180299218554, 805.39142381007036, 712.25447700734412, 630.84821673141755]

# relationships OXY (Jdf, N, S, Haro:)
# slopes:
# [-1.1093358620615621, -1.0324527832500823, -0.72610938589358809, -0.74540955174472656]
# intercepts:
# [39.482801413558569, 35.238673873674337, 25.996793064314204, 27.695179997365347]
# ox at 26

sn_nor_val = sn_nor[:]
DIC_nor_slope = 75.046682176725568
DIC_nor_int = -210.80778641253437

TA_nor_slope = 41.703959570158162
TA_nor_int = 805.39142381007036

OXY_nor_slope = -1.0324527832500823
OXY_nor_int = 35.238673873674337

DIC_nor = sn_nor_val*DIC_nor_slope + DIC_nor_int
TA_nor = sn_nor_val*TA_nor_slope + TA_nor_int
OXY_nor = sn_nor_val*OXY_nor_slope + OXY_nor_int

print('DIC at surface')
print(DIC_nor[:,:,0,0])
print('TA at surface')
print(TA_nor[:,:,0,0])
print('Oxygen at surface')
print(OXY_nor[:,:,0,0])

DIC at surface
[[ 2051.17431641  2052.90039062  2054.02612305  2055.15185547
   2056.27734375  2057.40307617  2058.52880859  2059.72949219
   2060.85522461  2061.98095703  2063.10668945  2064.23242188  2065.3581543
   2066.48364258  2067.68457031  2068.88525391  2070.16088867
   2071.58691406  2073.23803711  2075.41430664  2078.49121094
   2083.14404297  2090.42358398  2101.53051758  2117.36547852
   2137.85302734  2162.09326172  2167.94702148  2172.22460938
   2176.57739258  2181.00512695  2185.43286133  2189.93554688
   2194.43823242  2198.94116211  2203.44384766  2208.02172852  2212.5246582
   2217.02758789  2221.53027344]
 [ 2051.17431641  2052.90039062  2054.02612305  2055.15185547
   2056.27734375  2057.40307617  2058.52880859  2059.72949219
   2060.85522461  2061.98095703  2063.10668945  2064.23242188  2065.3581543
   2066.48364258  2067.68457031  2068.88525391  2070.16088867
   2071.58691406  2073.23803711  2075.41430664  2078.49121094
   2083.14404297  2090.42358398  2101.5305

In [9]:
no3 = tr_north.variables['NO3']
print(no3.shape)
print(DIC_nor.shape)

DIC_resized = no3[:]
TA_resized = no3[:]
OXY_resized = no3[:]


for i in range(0,10):
    start = i * 30 
    end = (i+1) * 30
    DIC_resized[:,:,:,start:end] = DIC_nor[:,:,i:i+1,:]
    TA_resized[:,:,:,start:end] = TA_nor[:,:,i:i+1,:]
    OXY_resized[:,:,:,start:end] = OXY_nor[:,:,i:i+1,:]
    
    
#print(DIC_resized[0,0,:,:])
#print(sn_nor.shape)
# print(DIC_nor[0,0,9,:])
# print(sn_nor[0,0,9,:])

print(no3.shape)

(2, 40, 1, 300)
(2, 40, 10, 30)
(2, 40, 1, 300)


In [10]:
dsout['DIC'][:] = DIC_resized
dsout['TA'][:] = TA_resized
dsout['OXY'][:] = OXY_resized


In [11]:
dsout.close()

In [12]:
w = nc.Dataset('./tr_northDICTAOXY.nc')

In [13]:
print(w)

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4 data model, file format HDF5):
    dimensions(sizes): xbT(300), yb(1), deptht(40), time_counter(2)
    variables(dimensions): float32 deptht(deptht), float32 nav_lat(yb,xbT), float32 nav_lon(yb,xbT), int32 nbidta(yb,xbT), int32 nbjdta(yb,xbT), int32 nbrdta(yb,xbT), float32 time_counter(time_counter), float32 NO3(time_counter,deptht,yb,xbT), float32 Si(time_counter,deptht,yb,xbT), float32 NH4(time_counter,deptht,yb,xbT), float32 DIA(time_counter,deptht,yb,xbT), float32 CRY(time_counter,deptht,yb,xbT), float32 MYRI(time_counter,deptht,yb,xbT), float32 MICZ(time_counter,deptht,yb,xbT), float32 PON(time_counter,deptht,yb,xbT), float32 DON(time_counter,deptht,yb,xbT), float32 bSi(time_counter,deptht,yb,xbT), float32 O2(time_counter,deptht,yb,xbT), float32 tur(time_counter,deptht,yb,xbT), float64 DIC(time_counter,deptht,yb,xbT), float64 TA(time_counter,deptht,yb,xbT), float64 OXY(time_counter,deptht,yb,xbT)
    groups: 

